In [32]:
import os
import sys
import time
import json

import urllib
import urllib.request
import requests

import pandas as pd
from bs4 import BeautifulSoup

import re

from tqdm import tqdm

In [10]:
data_path = r"C:\Users\Jwp\Desktop\workspace\Final_project\DataScience\Final_Data\movie\movieInfo\hot"
save_path = r"C:\Users\Jwp\Desktop\workspace\Final_project\DataScience\Final_Data\movie\movieReview"
# data_path = r"C:\Users\chongs\Desktop\git\final_project\DataScience\Final_Data\movie\movieInfo\all"
# save_path = r"C:\Users\chongs\Desktop\git\final_project\DataScience\Final_Data\movie\movieReview"
# data_path = r"D:\git_ec\final_project\DataScience\Final_Data\movie\movieInfo\all"
# save_path = r"D:\git_ec\final_project\DataScience\Final_Data\movie\movieReview"

In [13]:
movies = pd.read_csv(os.path.join(data_path, "movies_hot_merged_en.csv"), encoding="euc-kr")

In [41]:
movies[:5]

,movieCd,movieNm,movieNmEn,prdtYear,openDt,prdtStatNm,repGenre,genre,repNation,nations,...,directors,actors,actors_en,cast,cast_en,companyCd,companyNm,companyNmEn,companyPartNm,watchGradeNm
0,20090834,아바타,Avatar,2009,20091217,개봉,SF,"SF,액션,어드벤처",미국,미국,...,제임스 카메론,"['샘 워싱턴', '조 샐다나', '시고니 위버']","['Sam Worthington', 'Zoe Saldana', 'Sigourney ...","['', '', '']","['', '', '']","['20101507', '20175001', '20100041', '20175001...","['이십세기폭스필름코퍼레이션', '주식회사 해리슨앤컴퍼니', '이십세기폭스코리아(주...","['20th Century Fox Film Corporation', 'Harriso...","['제작사', '배급사', '배급사', '수입사', '수입사']","['12세이상관람가', '12세이상관람가']"
1,19980074,타이타닉,Titanic,1997,19980220,개봉,드라마,"드라마,멜로/로맨스,액션",미국,미국,...,제임스 카메론,"['레오나르도 디카프리오', '케이트 윈슬렛']","['Leonardo Dicaprio', 'Kate Winslet']","['', '']","['', '']","['20104469', '20040021', '20100041', '20142849...","['파라마운트 픽쳐스', '씨네힐', '이십세기폭스코리아(주)', '(주)영화사오원...","['Paramount Pictures', 'CINEHILL', '20th Centu...","['제작사', '배급사', '배급사', '수입사', '수입사']","['15세 미만인 자는 관람할 수 없는 등급 ', '15세이상관람가']"
2,20129370,명량,Roaring Currents,2013,20140730,개봉,사극,"사극,액션",한국,한국,...,김한민,"['최민식', '류승룡', '조진웅', '김명곤', '진구', '이정현', '권율'...","['CHOI Min-shik', 'RYU Seung-ryong', 'CHO Jin-...","['이순신 ', '구루지마 ', '와키자카 ', '도도', '임준영 ', '정씨 여...","['', '', '', '', '', '', '', '', '', '', '', '...","['20123063', '20110854', '20110854', '20100109']","['(주)빅스톤픽쳐스', '씨제이이앤엠(주)', '씨제이이앤엠(주)', '씨제이엔터...","['BIGSTONEPICTURES', 'CJ ENM Corp.', 'CJ ENM C...","['제작사', '배급사', '제공', '해외세일즈사']",['15세이상관람가']
3,20124079,"광해, 왕이 된 남자",Masquerade,2012,20120913,개봉,사극,"사극,드라마",한국,한국,...,추창민,"['이병헌', '류승룡', '한효주', '장광', '김인권', '심은경', '김명곤...","['LEE Byung-hun', 'RYU Seung-ryong', 'HAN Hyo-...","['광해/하선', '허균', '중전', '조내관', '도부장', '사월이', '박충...","['', '', '', '', '', '', '', '', '', '', '', '...","['20100540', '20110854', '20110854', '20110854...","['리얼라이즈픽쳐스(주)', '씨제이이앤엠(주)', '씨제이이앤엠(주)', '씨제이...","['Realies Pictures, Inc.', 'CJ ENM Corp.', 'CJ...","['제작사', '제작사', '배급사', '제공', '공동제공', '공동제공', '공...",['15세이상관람가']
4,20070061,트랜스포머,Transformers,2007,20070628,개봉,SF,"SF,액션,어드벤처",미국,미국,...,마이클 베이,"['샤이아 라보프', '메간 폭스']","['Shia LaBeouf', 'Megan Fox']","['샘 윗윅키', '']","['', '']","['20100694', '20102885', '20104469', '20100109...","['드림웍스픽쳐스', '드림웍스 SKG', '파라마운트 픽쳐스', '씨제이엔터테인먼...","['', 'DreamWorks SKG ', 'Paramount Pictures', ...","['제작사', '제작사', '제작사', '배급사', '수입사']",['12세이상관람가']


In [16]:
def progress_bar(i, total):
    k = i/total*100
    star = "*"*int(k/2.5)
    print("Processing: {}/{} [{:.1f}%] {}".format(i, total, k, star), end="\r")

In [37]:
def get_naverApi(movieNm, openDt, director):
    
    client_id = "JjcGyzzf5rjyFKNPCTxG"
    client_secret = "MDi_1o76AT"
    encText = urllib.parse.quote(f"{movieNm}")
    url = "https://openapi.naver.com/v1/search/movie?query=" + encText + f"&yearfrom={openDt-1}&yearto={openDt}"
    request = urllib.request.Request(url)

    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode == 200):
        html = response.read().decode("utf-8")
        parse = json.loads(html)
                
        if parse["items"] == []:
            link = ""
            
            
        else:
            if len(parse["items"]) > 1:
                for i, item in enumerate(parse["items"]):
                    if director in item["director"]:
                        idx = i
                    elif item["pubDate"] == str(openDt):
                        idx = i
                    else:
                        idx = 0
            else:
                 idx = 0
            
            info = parse["items"][idx]
            link = info["link"]
            
    else:
        link = ""
    
    return movieNm, link

In [38]:
def load_naverMovie(movie_list):
    movie_list.openDt = movie_list.openDt.astype(str)
    ls = movie_list[["movieCd","movieNm", "openDt", "directors"]].values
    
    i = 1
    total = len(ls)
    links = []
    movieNms = []
    movieCds = []
    for v in ls:
        movieNm, link = get_naverApi(v[1], int(v[2][:4]), v[3].split(" ")[0].split(",")[0])
        code = v[0]
        links.append(link)
        movieNms.append(movieNm)
        movieCds.append(code)
        progress_bar(i, total)
        i += 1
        time.sleep(0.1)

    print()
    print("complete!")
    
    return movieCds, movieNms, links

In [19]:
movieNm, link = get_naverApi("타이타닉", 1997, "제임스 카메론")

In [21]:
movieNm, link

('타이타닉', 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=18847')

In [121]:
naver_movie_code = link[51:]

In [39]:
def get_reviews(naver_movie_code):
    target_url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={naver_movie_code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'
    soup = BeautifulSoup(urllib.request.urlopen(target_url).read(), "html.parser")
    review_total = soup.find("div",class_="score_total")
    
#     review가 없는 경우는 아무것도 리턴하지 않는다
    if review_total is not None:
        number = review_total.find('em').text
        number = int(re.sub(',', '', number))
        page_num = number //10

        user_id = []
        review = []
        score = []
        
        for page in range(1, page_num):
            target_url_page = target_url + '&page={}'.format(page)
            soup = BeautifulSoup(urllib.request.urlopen(target_url_page).read(), "html.parser")
            review_divs = soup.find_all('div',class_='score_reple')
            score_divs = soup.find_all('div', class_="star_score")
            for spans in review_divs:
                txt = spans.find('p').text.strip()
                if txt is not None:
                    review.append(txt)
                else:
                    review.append('')
            for spans in review_divs:
                dl = spans.find('dl')
                if dl is not None:
                    user = dl.find('span').text
                    user_id.append(user)
                else:
                    user_id.append('')
            for span in score_divs:
                if span is not None:
                    s = span.find('em').text.strip()
                    score.append(s)
                else:
                    score.append('')
            
        return user_id, review, score
    else:
        return ["", "", ""]

In [73]:
movieCds, movieNms, links = load_naverMovie(movies[5:10])

Processing: 5/5 [100.0%] ****************************************
complete!


In [75]:
movieNms

['어벤져스: 엔드게임', '알라딘', '7번방의 선물', '겨울왕국', '어바웃 타임']

In [ ]:
user_id = []
review = []
score = []
for link in tqdm(links):
    code = link[51:]
    ids, reply, star_score = get_reviews(code)
    user_id.append(ids)
    review.append(reply)
    score.append(star_score)


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

In [58]:
len(user_id[0]), len(review[0]), len(score[0])

(40940, 40940, 40940)

In [ ]:
for i in range(5):
    df = pd.DataFrame([user_id[i], review[i], score[i]], index=["user_id", "review", "score"]).T
    df.to_csv(os.path.join(save_path, f"{movieCds[i]}_{movieNms[i]}.csv"), index=None, encoding="utf-8")

In [64]:
reviews = []
for i in range(5):
    reviews.append(pd.DataFrame([user_id[i], review[i], score[i]], index=["user_id", "review", "score"]).T)

In [72]:
for i in range(5):
    reviews[i].to_csv(os.path.join(save_path, f"{movies.iloc[i].movieCd} {movies.iloc[i].movieNm}.csv"), encoding="utf-8")